## load

In [201]:
import json

with open("conversations.json", "r", encoding="utf-8") as f:
    conversations = json.load(f)

with open("demographics.json", "r", encoding="utf-8") as f:
    demographics = json.load(f)

with open("nfr_responses.json", "r", encoding="utf-8") as f:
    nfr_responses = json.load(f)

with open("annotated_responses.json", "r", encoding="utf-8") as f:
    nfr_responses = json.load(f)

#with open("annotated_responses.json", "r", encoding="utf-8") as f:
#    annotated_responses = json.load(f)

with open("surveys.json", "r", encoding="utf-8") as f:
    surveys = json.load(f)

GT = {}
with open("../../GT/NFR.json", "r", encoding="utf-8") as f:
    for responses in json.load(f):
        for r in responses:
            GT[r['id']] = r





In [202]:
nfr_responses["406bdbef-755b-4032-a82f-86e358987177"] = nfr_responses["406bdbef-755b-4032-a82f-86e358987177"][:12]

review attention questions

In [203]:
nfr_responses_filtered = {}
for pid, responses in nfr_responses.items():
    nfr_responses_filtered[pid] = [r for r in responses if not r.get("is_attention_question", False)]

attention_questions = []
for pid, responses in nfr_responses.items():
    for r in responses:
        if r.get("is_attention_question", False):
            attention_questions.append(r)

In [204]:
print("Number of wrong attention questions in NFR list", len([r for r in attention_questions if r['nfr_acknowledged']== True]))
#[r for r in attention_questions if r['nfr_acknowledged']== True]

Number of wrong attention questions in NFR list 0


In [205]:
print("Number of wrong attention questions in feedback form", len([r for r in attention_questions if not (r['q1_agreement'] == 'Disagree' and r['q2_agreement'] == 'Partially disagree' and r['q3_agreement']== 'Partially agree')]))

Number of wrong attention questions in feedback form 0


## Analysis 1. (agreement)
We calculate the participants’ level of agreement by calculating the mean of their agreement

In [206]:
n = 0
satisfaction_level_agreement = 0
reasoning_agreement = 0
code_location_agreement = 0
agreement_value = {"Agree": 5, "Partially agree": 4, "Partially disagree": 2, "Disagree": 1}
for pid, responses in nfr_responses_filtered.items():
    for r in responses:
        n += 1
        satisfaction_level_agreement += agreement_value[r["q1_agreement"]]
        reasoning_agreement += agreement_value[r["q2_agreement"]]
        code_location_agreement += agreement_value[r["q3_agreement"]]

print("mean satisfaction level:", satisfaction_level_agreement / n)
print("mean reasoning agreement:", reasoning_agreement / n)
print("mean code location agreement:", code_location_agreement / n)

mean satisfaction level: 4.3
mean reasoning agreement: 4.3
mean code location agreement: 4.6


## Analysis 2. (accuracy of LLMs' evaluation of satisfaction level, reasoning, and code location)
We first extract the LLM's responses on satisfaction level, reasoning, and code location by manually reviewing the dialogues.
- For the satisfaction level, we calculate the F1 score between the extracted LLM response and the ground truth.
- For the reasoning, we measure accuracy as the mean of the similarity between the LLM’s response and the ground truth. We calculate this similarity using ROUGE or BERTScore.
- For code location, we compute the F1 score by defining true positives as |G ∩ P|, False Positives as |P \ G|, and False Negatives as |G \ P|, where G is the ground truth set, and P is the extracted Set.

### satisfaction level

In [207]:
y_true = []
y_pred = []
satisfavtion_levels = ["satisfied", "weakly satisfied", "weakly denied", "denied", "na"]
for pid, responses in nfr_responses_filtered.items():
    for response in responses:
        id = response['nfr_id']
        y1 = response["satisfaction_level"].lower()
        y2 = GT[id]["satisfaction_level"].lower()
        if y2 not in satisfavtion_levels:
            raise Exception(f'error {id} GT')
        if y1 not in satisfavtion_levels:
            raise Exception(f'error {id} {pid}')
        y_true.append(y2)
        y_pred.append(y1)


In [208]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred, average="macro")
print(f"Macro F1 score: {f1:.3f}")

Macro F1 score: 0.700


### reasoning

In [209]:
# pip install bert-score
# pip install transformers
# https://haticeozbolat17.medium.com/text-summarization-how-to-calculate-bertscore-771a51022964
#from transformers import BertTokenizer, BertForMaskedLM, BertModel
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
import torch
import numpy as np

# Example texts
reference = "This is a reference text example."
candidate = "This is a candidate text example."
# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')

def get_bert_score(reference, candidate):
    P, R, F1 = scorer.score([candidate], [reference])
    return P, R, F1

P, R, F1 = get_bert_score(reference, candidate)
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

def get_bert_cosine_sim(text1, text2):
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)
    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()
    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
    return similarity

text1 = "This is an example text."
text2 = "This text contains an example sentence."
similarity = get_bert_cosine_sim(text1, text2)
print("Similarity between the texts: {:.4f}".format(similarity[0][0]))

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1404.75it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BERTScore Precision: 0.9258, Recall: 0.9258, F1: 0.9258


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1653.22it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Similarity between the texts: 0.9000


In [ ]:
y_p = []
y_r = []
y_f = []
y_s = []
reasonings = []

for pid, responses in nfr_responses_filtered.items():
    for response in responses:
        id = response['nfr_id']
        y1 = response["reasoning"]
        y2 = GT[id]["reasoning"]
        P, R, F1 = get_bert_score(y1, y2)
        similarity = get_bert_cosine_sim(y1, y2)
        y_p.append(P.item())
        y_r.append(R.item())
        y_f.append(F1.item())
        y_s.append(similarity)
        reasonings.append((y1, y2))


print(f"BERTScore F1: {np.mean(y_f):.4f} ± {np.std(y_f):.4f}")
print(f"BERTScore P:  {np.mean(y_p):.4f} ± {np.std(y_p):.4f}")
print(f"BERTScore R:  {np.mean(y_r):.4f} ± {np.std(y_r):.4f}")
print(f"Cosine Sim:   {np.mean(y_s):.4f} ± {np.std(y_s):.4f}")

        

BERTScore F1: 0.5424 ± 0.0577
BERTScore P:  0.5234 ± 0.0440
BERTScore R:  0.5688 ± 0.0917
Cosine Sim:   0.7871 ± 0.0635


In [ ]:
import numpy as np

max_idx = np.argmax(y_s)
min_idx = np.argmin(y_s)


print("Highest similarity:")
print(y_s[max_idx])
print("y1:", reasonings[max_idx][0])
print("y2:", reasonings[max_idx][1])
print("\nLowest similarity:")
print(y_s[min_idx])
print("y1:", reasonings[min_idx][0])
print("y2:", reasonings[min_idx][1])

Highest similarity:
y1: form auth and role constraints guard /auth/* resources in webroot/web-inf/web.xml lines 35-314 and authdao.authenticatepassword uses salted hashes to admit only valid credentials (lines 503-513);
y2: (1) users must log in with username and password (form login in web.xml). (2) all ephi lives under /auth/*; those urls require a valid login. so access is further restricted by role and url path in web.xml. (3) ephi is not only “by role” but also “this user can only see this patient’s data.” for example, in viewmyrecordsaction, the code uses the user’s id so they only get their own record.

Lowest similarity:
y1: emergencyrecordcontroller/emergencyrecordmysql load emergency data and log emergency_report_view.
y2: emergency responders use a dedicated er role for viewing patient records during emergencies.


### code locations

In [ ]:
y_p = []
y_r = []
y_f = []
y_s = []
reasonings = []

for pid, responses in nfr_responses_filtered.items():
    for response in responses:
        id = response['nfr_id']
        # TODO
        y1 = ";".join(response["code_location"])
        y2 = ";".join(GT[id]["code_location"])
        if not y1 and not y2:
            print('both y1 and y2 are empty skipping')
            continue
        if not y1:
            print('y1 is empty skipping')
            continue
        if not y2:
            print('y2 is empty skipping')
            continue        
        P, R, F1 = get_bert_score(y1, y2)
        similarity = get_bert_cosine_sim(y1, y2)
        y_p.append(P.item())
        y_r.append(R.item())
        y_f.append(F1.item())
        y_s.append(similarity)
        reasonings.append((y1, y2))

print()
print(f"BERTScore F1: {np.mean(y_f):.4f} ± {np.std(y_f):.4f}")
print(f"BERTScore P:  {np.mean(y_p):.4f} ± {np.std(y_p):.4f}")
print(f"BERTScore R:  {np.mean(y_r):.4f} ± {np.std(y_r):.4f}")
print(f"Cosine Sim:   {np.mean(y_s):.4f} ± {np.std(y_s):.4f}")

both y1 and y2 are empty skipping
both y1 and y2 are empty skipping
BERTScore F1: 0.6181 ± 0.1346
BERTScore P:  0.6336 ± 0.1420
BERTScore R:  0.6144 ± 0.1537
Cosine Sim:   0.7673 ± 0.1597


In [ ]:
import numpy as np

max_idx = np.argmax(y_s)
min_idx = np.argmin(y_s)


print("Highest similarity:")
print(y_s[max_idx])
print("y1:", reasonings[max_idx][0])
print("y2:", reasonings[max_idx][1])
print("\nLowest similarity:")
print(y_s[min_idx])
print("y1:", reasonings[min_idx][0])
print("y2:", reasonings[min_idx][1])

## Analysis 3.(significant dialogue costs)
To determine which dialogue metrics (Table 1) significantly correlate with dialogue performance, we first manually review the dialogues to extract dialogue cost values. For example, the mean elapsed time is calculated by dividing the difference between the first and last message submission times in the chatbot by the number of tasks assigned to the user. Or, task completion is 1 when satisfaction level, reasoning, and code location are identified in the dialogues; otherwise, it's 0. The number of user initiatives is the number of times a user doesn't ask a follow-up question to the previous message, and so on. For task success, we calculate Cohen's kappa between the LLM's response and the ground truth. We then measure user satisfaction by summing the Likert scores in the post-study survey questions (Table 2). Based on the Paradise framework, the performance of a dialogue system (i.e., the prediction of user satisfaction) is modeled as a multiple linear regression of dialogue costs (independent variables). We can determine whether there is any significant relationship between the performance and any of the dialogue costs. A common procedure for testing the significance of variables in a multiple linear regression model is to run an 'Omnibus' test followed by 'Post-Hoc' tests.


In [ ]:
for pid, dialogues in conversations.items():
    num_tasks = len(nfr_responses_filtered)
    MET = (dialogues[-1]["bot_time"] - dialogues[0]["user_time"]) / num_tasks
    MUM = len(dialogues) / num_tasks
    Comp = 1 # TODO
    MWT = 0
    MRT = 0
    NMR = 0 #
    NUR = 0
    Error = 0
    for dialogue in dialogues:
        MWT += dialogue["user_message"]
        MRT += dialogue["bot_reply"]
        if dialogue.get("NUR"):
            NUR += 1
    MWT = MWT / len(dialogue)
    MRT = MRT / len(dialogue)
    URR = NUR / num_tasks

    Task_Success = 1 # TODO


